# Predicciones multivariadas


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pylab import rcParams
from sklearn.model_selection import ParameterGrid
import itertools
import warnings
import seaborn as sns
from dateutil.relativedelta import relativedelta
import time



#Own packages
import download
import descriptive
import models_multivariate as mv
import grid

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings("ignore")

## Importamos datos

In [3]:
# Importamos csv con ingresos netos
ingresos_netos = download.load_ingresos_fiscales_netos()

# Balanza comercial
importaciones = download.load_balanza_comercial('importaciones')
exportaciones=download.load_balanza_comercial('exportaciones')

# Indicadores
igae = download.load_inegi_indic('igae')
igae_sa = download.load_inegi_indic('igae_sa')
igae_prim=download.load_inegi_indic('igae_prim')
igae_secun=download.load_inegi_indic('igae_secun')
igae_terc=download.load_inegi_indic('igae_terc')
indic_mens_consumo = download.load_inegi_indic('indic_mens_consumo')
confianza_consumidor=download.load_inegi_indic('confianza_consumidor')
indic_mens_consumo=download.load_inegi_indic('indic_mens_consumo')
indic_adelant=download.load_inegi_indic('indic_adelant')

# Imai
imai = download.load_inegi_indic('imai')
imai_mineria=download.load_inegi_indic('imai_mineria')
imai_construccion=download.load_inegi_indic('imai_construccion')
imai_manufacturas=download.load_inegi_indic('imai_manufacturas')
imai_egergia_gas_agua_gas=download.load_inegi_indic('imai_egergia_gas_agua_gas')


# Ocupación
asegurados_imss = download.load_inegi_indic('asegurados_imss')
pea=download.load_inegi_indic('pea')
pobl_ocupada=download.load_inegi_indic('pobl_ocupada')

# Tipo de cambio
tc_mensual = download.load_banxico('tc_mensual')
trade_weighted_exchange_rate = download.load_banxico('trade_weighted_exchange_rate')

# Tasas de interés
tasa_cetes_91_mensual = download.load_banxico('tasa_cetes_91_mensual')
tbill_3meses_mensual = download.load_fed('tbill_3meses_mensual')

# Precios
cons_price_index_us = download.load_fed('cons_price_index_us')
inpc = download.load_inpc()
commodity_price_index = download.load_fed('commodity_price_index')

# Indicadores US
ind_prod_ind_us = download.load_fed('ind_prod_ind_us')

# Semana santa
semana_santa = pd.read_excel('../inputs/semana_santa.xlsx', index_col='fecha')
semana_santa['semana_santa'] = 1
semana_santa = semana_santa.reindex(pd.date_range('1990-01-01', '2022-12-01', freq='MS'))
semana_santa = semana_santa.fillna(0)

# Reformas tributarias
tasas_impuestos = pd.read_excel('../inputs/tasas_impuestos.xlsx', index_col='fecha')
tasas_impuestos = tasas_impuestos.diff(12)

# Semana santa fin de marzo
semana_santa_fin_marzo = pd.read_excel('../inputs/semana_santa_fin_mazo.xlsx', index_col='fecha')
semana_santa_fin_marzo['semana_santa'] = 1
semana_santa_fin_marzo = semana_santa.reindex(pd.date_range('1990-01-01', '2022-12-01', freq='MS'))
semana_santa_fin_marzo = semana_santa.reindex(pd.date_range('1990-01-01', '2022-12-01', freq='MS'))

# Construimos df con semana santa y tasas tribtarias
semana_santa_y_tasas = pd.concat([semana_santa_fin_marzo, tasas_impuestos], axis=1)
semana_santa_y_tasas = semana_santa_y_tasas.loc[semana_santa_y_tasas.notna().all(1)]

In [48]:
params = {
    'beginings': ['2015-07-01', '2016-07-01', '2017-07-01', '2018-07-01'],
#     'beginings': ['2018-07-01'],
    'periods': 18,
    'econometric_models': 'medium',
    'ml_models': 'custom',
    'us_economy_model':{
        'endog_vars': ['tbill_3meses_mensual', 'cons_price_index_us',\
                      'ind_prod_ind_us', 'trade_weighted_exchange_rate',
                      'commodity_price_index'],
        'endog_transformation': 'log_diff',
        'year_dummies': [2000, 2009],
        'start_var': '1992-01-01'},
    'mx_economy_model':[
            {'endog_vars': ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc',\
                           'ing_trib_sin_gasol_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'start_var': '2000-01-01'},
            {'endog_vars': ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc',\
                           'ing_trib_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'start_var': '2000-01-01'},
            {'endog_vars': ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc',\
                           'importaciones_r', 'indic_mens_consumo'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'start_var': '2000-01-01'}],
    'ingresos_fiscales_var':[
            {'endog_vars': ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'exog_vars': ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
                          'tc_mensual', 'commodity_price_index', 'importaciones_r', 'indic_mens_consumo'],
             'other_exog': semana_santa_y_tasas,
             'month_dummies': [1, 3, 4, 12],
             'start_var': '2000-01-01'},
            {'endog_vars': ['isr_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'exog_vars': ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
                          'tc_mensual', 'commodity_price_index', 'importaciones_r', 'indic_mens_consumo'],
             'other_exog': semana_santa_y_tasas,
             'month_dummies': [1, 3, 4, 12],
             'start_var': '2000-01-01'},
            {'endog_vars': ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'exog_vars': ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
                          'tc_mensual', 'commodity_price_index', 'importaciones_r', 'indic_mens_consumo'],
             'other_exog': semana_santa_y_tasas,
             'month_dummies': [1, 3, 4, 12],
             'start_var': '2000-01-01'},
            {'endog_vars': ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'ieps_sin_gas_neto_(mdp)_r',
                            'importaciones_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'exog_vars': ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
                           'tc_mensual', 'commodity_price_index', 'importaciones_r', 'indic_mens_consumo'],
             'other_exog': semana_santa_y_tasas,
             'month_dummies': [1, 3, 4, 12],
             'start_var': '2000-01-01'}],
    'ingresos_fiscales_arima': [
            {'models': ['ARIMA', 'SARIMA'],
             'endog_vars': ['ing_trib_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'year_dummies': [2009, 2015],
             'exog_transformation': 'log_diff',
             'month_dummies': [1, 3, 4, 12],
             'exog_vars': [False, ['igae_sa'], ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
                                            'tc_mensual', 'commodity_price_index',
                                            'importaciones_r', 'indic_mens_consumo']],
             'start_arima': '1990-01-01'}],
#             {'models': ['ARIMA', 'SARIMA'],
#              'endog_vars': ['isr_neto_(mdp)_r'],
#              'endog_transformation': 'log_diff',
#              'year_dummies': [2009, 2015],
#              'exog_transformation': 'log_diff',
#              'month_dummies': [1, 3, 4, 12],
#              'exog_vars': [False, ['igae_sa'], ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
#                                             'tc_mensual', 'commodity_price_index',
#                                             'importaciones_r', 'indic_mens_consumo']],
#              'start_arima': '1990-01-01'},
#             {'models': ['ARIMA', 'SARIMA'],
#              'endog_vars': ['iva_neto_(mdp)_r'],
#              'endog_transformation': 'log_diff',
#              'year_dummies': [2009, 2015],
#              'exog_transformation': 'log_diff',
#              'month_dummies': [1, 3, 4, 12],
#              'exog_vars': [False, ['igae_sa'], ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
#                                             'tc_mensual', 'commodity_price_index',
#                                             'importaciones_r', 'indic_mens_consumo']],
#              'start_arima': '1990-01-01'},
#             {'models': ['ARIMA', 'SARIMA'],
#              'endog_vars': ['ing_trib_sin_gasol_neto_(mdp)_r'],
#              'endog_transformation': 'log_diff',
#              'year_dummies': [2009, 2015],
#              'exog_transformation': 'log_diff',
#              'month_dummies': [1, 3, 4, 12],
#              'exog_vars': [False, ['igae_sa'], ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
#                                             'tc_mensual', 'commodity_price_index',
#                                             'importaciones_r', 'indic_mens_consumo']],
#              'start_arima': '1990-01-01'},
#             {'models': ['ARIMA', 'SARIMA'],
#              'endog_vars': ['importaciones_neto_(mdp)_r'],
#              'endog_transformation': 'log_diff',
#              'year_dummies': [2009, 2015],
#              'exog_transformation': 'log_diff',
#              'month_dummies': [1, 3, 4, 12],
#              'exog_vars': [False, ['importaciones_r'], ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
#                                             'tc_mensual', 'commodity_price_index',
#                                             'importaciones_r', 'indic_mens_consumo']],
#              'start_arima': '1990-01-01'},
#             {'models': ['ARIMA', 'SARIMA'],
#              'endog_vars': ['ieps_sin_gas_neto_(mdp)_r'],
#              'endog_transformation': 'log_diff',
#              'year_dummies': [2009, 2015],
#              'exog_transformation': 'log_diff',
#              'month_dummies': [1, 3, 4, 12],
#              'exog_vars': [False, ['igae_sa'], ['igae_sa', 'inpc', 'tasa_cetes_91_mensual',\
#                                             'tc_mensual', 'commodity_price_index',
#                                             'importaciones_r', 'indic_mens_consumo']],
#              'start_arima': '1990-01-01'}],
        'ml_model':[
#             {'models': ['GB', 'RF'],
#              'endog_vars': ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'ing_trib_sin_gasol_neto_(mdp)_r',
#                             'ieps_sin_gas_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'],
#              'endog_transformation': 'log_diff',
#              'endog_lags': range(1, 13),
#              'year_dummies': True,
#              'exog_transformation': 'log_diff',
#              'exog_pct_changes': [1, 12],
#              'exog_lags': range(1, 13),
#              'month_dummies': True,
#              'start': '1990-01-01'},
            {'models': ['GB'],
             'endog_vars': ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'},           
            {'models': ['GB'],
             'endog_vars': ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'},
            {'models': ['GB', 'RF'],
             'endog_vars': ['iva_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'},
            {'models': ['GB'],
             'endog_vars': ['isr_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'},
            {'models': ['GB'],
             'endog_vars': ['ing_trib_sin_gasol_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'},
            {'models': ['GB'],
             'endog_vars': ['ieps_sin_gas_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'},
            {'models': ['GB'],
             'endog_vars': ['importaciones_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'},
            {'models': ['GB'],
             'endog_vars': ['ing_trib_neto_(mdp)_r'],
             'endog_transformation': 'log_diff',
             'endog_lags': range(1, 13),
             'year_dummies': True,
             'exog_transformation': 'log_diff',
             'exog_pct_changes': [1, 12],
             'exog_lags': range(1, 13),
             'month_dummies': True,
             'start': '1990-01-01'}]
}

params['models'] = {**grid.econometric_models[params['econometric_models']], **grid.ml_models[params['ml_models']]}

In [5]:
all_models_params = {}
for model, specifications in params['models'].items():
    all_models_params[model] = list(ParameterGrid(specifications))

In [6]:
all_models_params

{'ARIMA': [{'order': (2, 0, 2)},
  {'order': (2, 0, 3)},
  {'order': (2, 0, 4)},
  {'order': (3, 0, 2)},
  {'order': (3, 0, 3)},
  {'order': (3, 0, 4)},
  {'order': (4, 0, 2)},
  {'order': (4, 0, 3)},
  {'order': (4, 0, 4)}],
 'SARIMA': [{'enforce_invertibility': False,
   'enforce_stationarity': False,
   'order': (2, 0, 2),
   'seasonal_order': (0, 0, 0, 12)},
  {'enforce_invertibility': False,
   'enforce_stationarity': False,
   'order': (2, 0, 2),
   'seasonal_order': (0, 0, 1, 12)},
  {'enforce_invertibility': False,
   'enforce_stationarity': False,
   'order': (2, 0, 2),
   'seasonal_order': (1, 0, 0, 12)},
  {'enforce_invertibility': False,
   'enforce_stationarity': False,
   'order': (2, 0, 2),
   'seasonal_order': (1, 0, 1, 12)},
  {'enforce_invertibility': False,
   'enforce_stationarity': False,
   'order': (2, 0, 3),
   'seasonal_order': (0, 0, 0, 12)},
  {'enforce_invertibility': False,
   'enforce_stationarity': False,
   'order': (2, 0, 3),
   'seasonal_order': (0, 0,

In [7]:
# Nos quedamos unicamente ingresos reales
ingresos_reales = ['ieps_neto_(mdp)_r', 'ieps_sin_gas_neto_(mdp)_r', 
                   'ing_gob_fed_neto_(mdp)_r', 'ing_trib_neto_(mdp)_r', 'isr_neto_(mdp)_r',
                   'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r', 'ing_trib_sin_gasol_neto_(mdp)_r']

ingresos_netos_reales = ingresos_netos[ingresos_reales]

## Construimos DF conjunto

In [8]:
# Primero construniso DF de USA
df_netos_us = pd.concat([tbill_3meses_mensual, cons_price_index_us, ind_prod_ind_us, trade_weighted_exchange_rate,
                         commodity_price_index],
                        axis=1)

df_netos_us = df_netos_us.asfreq('MS')
df_netos_us = df_netos_us.loc['1992-01-01': '2019-06-01']

In [9]:
# Vamos cuales son las variables que tenemos que hacer nowcast
vars_to_predict_us = df_netos_us.columns[df_netos_us.tail().isna().any()]
for var in vars_to_predict_us:
    # Volvemos a obtener variables que tienen valores nulos, pues cada que predecimos una variable,
    # acutalizamos.
    empty_vars = df_netos_us.columns[df_netos_us.tail().isna().any()]
    us_vars_to_use_as_covars = [var for var in df_netos_us.columns if var not in empty_vars]
    exog = df_netos_us[us_vars_to_use_as_covars].copy()
    # Creamos variable de mes
    exog = mv.construct_exog(exog, month_dummies=True, year_dummies=[2000, 2009])
    prediction_start = df_netos_us.loc[df_netos_us[var].notna()].index.max() + relativedelta(months=1)
    _, prediction = mv.predict_with_econometric_model(
        model_name='SARIMA',
        params={'order': (3, 0, 3), 'seasonal_order': (1, 0, 1, 12)},
        df=df_netos_us,
        endog_vars=[var],
        prediction_start=prediction_start,
        prediction_end=df_netos_us.index.max(),
        transformation='log_diff',
        lags_endog=None,
        exog_df=exog,
        train_start=None)
    df_netos_us.loc[pd.date_range(prediction_start, df_netos_us.index.max(), freq='MS'), var] = prediction
    print('Nowcasted {}'.format(var))

Nowcasted commodity_price_index


In [10]:
print(time.ctime())

Tue Aug 13 21:25:58 2019


In [11]:
# Ahora construimos DF conjunto
df_netos = pd.concat([ingresos_netos_reales, igae_sa, tc_mensual, tasa_cetes_91_mensual,
                      imai, inpc, df_netos_us, importaciones, indic_mens_consumo], axis=1)
df_netos = df_netos.asfreq('MS')
# Filtramos df para post 2000
df_netos = df_netos.loc['1993-01-01':'2019-06-01']

In [12]:
# Vamos cuales son las variables que tenemos que hacer nowcast
vars_to_predict = df_netos.columns[df_netos.tail().isna().any()]
vars_to_predict

Index(['igae_sa', 'indic_mens_consumo'], dtype='object')

In [13]:
vars_to_predict = df_netos.columns[df_netos.tail().isna().any()]
for var in vars_to_predict:
    # Volvemos a obtener variables que tienen valores nulos, pues cada que predecimos una variable,
    # acutalizamos.
    empty_vars = df_netos.columns[df_netos.tail().isna().any()]
    vars_to_use_as_covars = [var for var in df_netos.columns if var not in empty_vars]
    exog = df_netos[vars_to_use_as_covars].copy()
    # Creamos variable de mes
    exog = mv.construct_exog(exog, month_dummies=True, year_dummies=[2000, 2009])
    prediction_start = df_netos.loc[df_netos[var].notna()].index.max() + relativedelta(months=1)
    _, prediction = mv.predict_with_econometric_model(
        model_name='SARIMA',
        params={'order': (3, 0, 3), 'seasonal_order': (1, 0, 1, 12),
                'enforce_stationarity': False, 'enforce_invertibility': False},
        df=df_netos,
        endog_vars=[var],
        prediction_start=prediction_start,
        prediction_end=df_netos.index.max(),
        transformation='log_diff',
        lags_endog=None,
        exog_df=exog,
        train_start=None)
    df_netos.loc[pd.date_range(prediction_start, df_netos.index.max(), freq='MS'), var] = prediction
    print('Nowcasted {}'.format(var))

Nowcasted igae_sa
Nowcasted indic_mens_consumo


In [14]:
print(time.ctime())

Tue Aug 13 21:26:01 2019


In [15]:
begin_and_ends = mv.get_start_and_end_dates(params['beginings'], params['periods'])

In [16]:
# Predecimos US
# Obtenemos predicciones y resultados
prediction_us, accuracies_us, _ = mv.run_predictions(
                                    model_name='VAR',
                                    params=all_models_params['VAR'],
                                    begin_and_ends=begin_and_ends,
                                    df=df_netos_us,
                                    endog_cols=params['us_economy_model']['endog_vars'],
                                    endog_transformation=params['us_economy_model']['endog_transformation'],
                                    endog_lags=None,
                                    us_prediction_dict=None,
                                    exog=True,
                                    mex_prediction_dict=None,
                                    other_exog=None,
                                    exog_pct_changes=None,
                                    exog_month_dummies=None,
                                    exog_year_dummies=params['us_economy_model']['year_dummies'],
                                    exog_lags=None,
                                    train_start=params['us_economy_model']['start_var'])
# Vemos resultados y nos quedamos con la mejor especificación.
us_var_accuracies = pd.DataFrame(accuracies_us)
us_var_accuracies.to_csv('../results/results_var_us_new.csv')
us_var_accuracies.groupby('params').mean().sort_values('mape_first6')

COMPLETED VAR for ['tbill_3meses_mensual', 'cons_price_index_us', 'ind_prod_ind_us', 'trade_weighted_exchange_rate', 'commodity_price_index'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['tbill_3meses_mensual', 'cons_price_index_us', 'ind_prod_ind_us', 'trade_weighted_exchange_rate', 'commodity_price_index'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['tbill_3meses_mensual', 'cons_price_index_us', 'ind_prod_ind_us', 'trade_weighted_exchange_rate', 'commodity_price_index'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['tbill_3meses_mensual', 'cons_price_index_us', 'ind_prod_ind_us', 'trade_weighted_exchange_rate', 'commodity_price_index'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['tbill_3meses_mensual', 'cons_price_index_us', 'ind_prod_ind_us', 'trad

,forecast_biass_first18,forecast_biass_first6,forecast_biass_last12,mae_first18,mae_first6,mae_last12,mape_first18,mape_first6,mape_last12,rmse_first18,rmse_first6,rmse_last12
params,,,,,,,,,,,,
"{'ic': 'bic', 'maxlags': 12, 'trend': 'ct'}",1.473068,0.137642,2.068345,4.100789,2.649123,4.888720,0.128683,0.076058,0.155678,4.727877,3.097808,5.284433
"{'ic': 'bic', 'maxlags': 12, 'trend': 'c'}",-0.277646,-0.523798,-0.284982,3.870182,2.644663,4.578382,0.132141,0.077355,0.161471,4.382072,3.089228,4.906897
"{'ic': 'aic', 'maxlags': 12, 'trend': 'ct'}",1.627080,-0.143196,2.517295,3.450526,3.372475,3.496102,0.129288,0.100423,0.148609,3.981318,3.691163,3.975718
"{'ic': 'fpe', 'maxlags': 12, 'trend': 'ct'}",1.627080,-0.143196,2.517295,3.450526,3.372475,3.496102,0.129288,0.100423,0.148609,3.981318,3.691163,3.975718
"{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}",-1.595687,-1.470514,-1.798273,4.222222,4.061005,4.411591,0.135292,0.106930,0.150198,4.772864,4.418740,4.824184
"{'ic': 'fpe', 'maxlags': 12, 'trend': 'c'}",-1.595687,-1.470514,-1.798273,4.222222,4.061005,4.411591,0.135292,0.106930,0.150198,4.772864,4.418740,4.824184


In [17]:
accuracies_us

[{'model': 'VAR',
  'params': "{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}",
  'split_date': Timestamp('2015-07-01 00:00:00'),
  'pred_period': '18MS',
  'exog_vars': "['year_2000', 'year_2009']",
  'endog_vars': "['tbill_3meses_mensual', 'cons_price_index_us', 'ind_prod_ind_us', 'trade_weighted_exchange_rate', 'commodity_price_index']",
  'variable': 'tbill_3meses_mensual',
  'rmse_first6': 0.10555119797211883,
  'mae_first6': 0.07106125528228847,
  'mape_first6': 0.6910133884503029,
  'forecast_biass_first6': 0.07106125528228847,
  'rmse_last12': 0.3212626878237851,
  'mae_last12': 0.31161271031597426,
  'mape_last12': 0.9802939188231168,
  'forecast_biass_last12': 0.31161271031597426,
  'rmse_first18': 0.269295676762473,
  'mae_first18': 0.23142889197141234,
  'mape_first18': 0.883867075365512,
  'forecast_biass_first18': 0.23142889197141234},
 {'model': 'VAR',
  'params': "{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}",
  'split_date': Timestamp('2015-07-01 00:00:00'),
  'pred_period':

In [18]:
best_specification = "{'ic': 'bic', 'maxlags': 12, 'trend': 'c'}"
best_prediction_us = {}
for split, predictions in prediction_us.items():
    best_prediction_us[split] = predictions[best_specification]

In [59]:
best_prediction_us.keys()

dict_keys([Timestamp('2015-07-01 00:00:00'), Timestamp('2016-07-01 00:00:00'), Timestamp('2017-07-01 00:00:00'), Timestamp('2018-07-01 00:00:00')])

In [19]:
print(time.ctime())

Tue Aug 13 21:26:03 2019


# Modelo VAR para economía Mexicana
- Opción1: Incluyendo ingresos tributarios sin gasolina
- Opción2: No incluyendo ingresos tributarios sin gasolina

## Opción 1.1: Incluyendo ingresos tributarios_sin_ieps_gas_netos

In [20]:
# Predecimos Mexico
prediction_mx_w_trib, accuracies_mx_w_trib, y_mx_diffs_w_trib = mv.run_predictions(
    model_name='VAR',
    params=all_models_params['VAR'],
    begin_and_ends=begin_and_ends,
    df=df_netos,
    endog_cols=params['mx_economy_model'][0]['endog_vars'],
    endog_transformation=params['mx_economy_model'][0]['endog_transformation'],
    endog_lags=None,
    exog=True,
    exog_transformation=params['mx_economy_model'][0]['exog_transformation'], 
    mex_prediction_dict=None,
    us_prediction_dict=best_prediction_us,
    other_exog=semana_santa_y_tasas,
    exog_pct_changes=None,
    exog_month_dummies=None,
    exog_year_dummies=params['mx_economy_model'][0]['year_dummies'],
    exog_lags=None,
    train_start=params['mx_economy_model'][0]['start_var'])
# Vemos resultados y nos quedamos con la mejor especificación.
mx_var_w_trib_accuracies = pd.DataFrame(accuracies_mx_w_trib)
mx_var_w_trib_accuracies.to_csv('../results/results_var_mx_w_trib_new.csv')

COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_sin_gasol_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_sin_gasol_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_sin_gasol_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_sin_gasol_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 't

In [21]:
print(time.ctime())

Tue Aug 13 21:26:07 2019


In [22]:
mx_var_w_trib_accuracies.loc[mx_var_w_trib_accuracies['variable'] == 'ing_trib_sin_gasol_neto_(mdp)_r']\
    .sort_values('mape_first6').head()

,endog_vars,exog_vars,forecast_biass_first18,forecast_biass_first6,forecast_biass_last12,mae_first18,mae_first6,mae_last12,mape_first18,mape_first6,mape_last12,model,params,pred_period,rmse_first18,rmse_first6,rmse_last12,split_date,variable
46,"['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensu...","['commodity_price_index', 'cons_price_index_us...",3866.555762,9341.592922,1129.037182,15784.968054,13184.261341,17085.321410,0.064860,0.054725,0.069927,VAR,"{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}",18MS,20091.956729,16913.162503,21505.872566,2016-07-01,ing_trib_sin_gasol_neto_(mdp)_r
130,"['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensu...","['commodity_price_index', 'cons_price_index_us...",-3687.966204,-89.662018,-7286.270391,12449.467601,13695.865891,11203.069311,0.049817,0.057622,0.042011,VAR,"{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}",18MS,14423.936906,15312.126399,13477.340126,2018-07-01,ing_trib_sin_gasol_neto_(mdp)_r
158,"['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensu...","['commodity_price_index', 'cons_price_index_us...",-3687.966204,-89.662018,-7286.270391,12449.467601,13695.865891,11203.069311,0.049817,0.057622,0.042011,VAR,"{'ic': 'fpe', 'maxlags': 12, 'trend': 'c'}",18MS,14423.936906,15312.126399,13477.340126,2018-07-01,ing_trib_sin_gasol_neto_(mdp)_r
165,"['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensu...","['commodity_price_index', 'cons_price_index_us...",75.987733,1506.011783,-1354.036316,11218.685853,13982.493598,8454.878109,0.044743,0.058899,0.030588,VAR,"{'ic': 'fpe', 'maxlags': 12, 'trend': 'ct'}",18MS,13887.077219,16003.524469,11383.717845,2018-07-01,ing_trib_sin_gasol_neto_(mdp)_r
137,"['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensu...","['commodity_price_index', 'cons_price_index_us...",75.987733,1506.011783,-1354.036316,11218.685853,13982.493598,8454.878109,0.044743,0.058899,0.030588,VAR,"{'ic': 'aic', 'maxlags': 12, 'trend': 'ct'}",18MS,13887.077219,16003.524469,11383.717845,2018-07-01,ing_trib_sin_gasol_neto_(mdp)_r


In [23]:
best_specification_mx_w_trib = "{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}"
best_prediction_mx_w_trib = {}
for split, predictions in prediction_mx_w_trib.items():
    best_prediction_mx_w_trib[split] = predictions[best_specification_mx_w_trib]

In [24]:
y_mx_diffs_w_trib_df = pd.DataFrame(y_mx_diffs_w_trib)
y_mx_diffs_w_trib_df.to_csv('../results/precision_cierre_mx_var_w_trib.csv')

## Opción 1.2: Incluyendo ingresos tributarios_netos

In [49]:
# Predecimos Mexico
prediction_mx_w_trib_tot, accuracies_mx_w_trib_tot, y_mx_diffs_w_trib_tot = mv.run_predictions(
    model_name='VAR',
    params=all_models_params['VAR'],
    begin_and_ends=begin_and_ends,
    df=df_netos,
    endog_cols=params['mx_economy_model'][1]['endog_vars'],
    endog_transformation=params['mx_economy_model'][1]['endog_transformation'],
    endog_lags=None,
    exog=True,
    exog_transformation=params['mx_economy_model'][1]['exog_transformation'], 
    mex_prediction_dict=None,
    us_prediction_dict=best_prediction_us,
    other_exog=semana_santa_y_tasas,
    exog_pct_changes=None,
    exog_month_dummies=None,
    exog_year_dummies=params['mx_economy_model'][1]['year_dummies'],
    exog_lags=None,
    train_start=params['mx_economy_model'][1]['start_var'])
# Vemos resultados y nos quedamos con la mejor especificación.
mx_var_w_trib_tot_accuracies = pd.DataFrame(accuracies_mx_w_trib_tot)
mx_var_w_trib_tot_accuracies.to_csv('../results/results_var_mx_w_trib_tot_new.csv')

COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib_neto_(mdp)_r', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'ing_trib

In [50]:
y_mx_diffs_w_trib_tot_df = pd.DataFrame(y_mx_diffs_w_trib_tot)
y_mx_diffs_w_trib_tot_df.to_csv('../results/precision_cierre_mx_var_w_trib_tot.csv')

## Opción 2: No Incluyendo ingresos tributarios

In [25]:
# Predecimos Mexico
prediction_mx_no_trib, accuracies_mx_no_trib, y_mx_diffs_no_trib = mv.run_predictions(
    model_name='VAR',
    params=all_models_params['VAR'],
    begin_and_ends=begin_and_ends,
    df=df_netos,
    endog_cols=params['mx_economy_model'][1]['endog_vars'],
    endog_transformation=params['mx_economy_model'][1]['endog_transformation'],
    endog_lags=None,
    exog=True,
    exog_transformation=params['mx_economy_model'][1]['exog_transformation'], 
    mex_prediction_dict=None,
    us_prediction_dict=best_prediction_us,
    other_exog=semana_santa,
    exog_pct_changes=None,
    exog_month_dummies=None,
    exog_year_dummies=params['mx_economy_model'][1]['year_dummies'],
    exog_lags=None,
    train_start=params['mx_economy_model'][1]['start_var'])
# Vemos resultados y nos quedamos con la mejor especificación.
mx_var_no_trib_accuracies = pd.DataFrame(accuracies_mx_no_trib)
mx_var_no_trib_accuracies.to_csv('../results/results_var_mx_no_trib_new.csv')

COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
VAR model selected lags 0, for params {'ic': 'bic', 'maxlags': 12, 'trend': 'c'} and split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
VAR model selected lags 0, for params {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} and split 2015-07-01 00:00:00
COMPLETED VAR for ['igae_sa', 'tc_mensual', 'tasa_cetes_91_mensual', 'inpc', 'importaciones_r', 'indic_mens_consumo'] with params {'ic': 'bic', 'maxlags': 12, 'trend':

In [26]:
y_mx_diffs_no_trib_df = pd.DataFrame(y_mx_diffs_no_trib)
y_mx_diffs_no_trib_df.to_csv('../results/precision_cierre_mx_var_no_trib.csv')

In [27]:
print(time.ctime())

Tue Aug 13 21:26:10 2019


In [28]:
mx_var_no_trib_accuracies.groupby('params').mean().sort_values('mape_first6').head()

,forecast_biass_first18,forecast_biass_first6,forecast_biass_last12,mae_first18,mae_first6,mae_last12,mape_first18,mape_first6,mape_last12,rmse_first18,rmse_first6,rmse_last12
params,,,,,,,,,,,,
"{'ic': 'bic', 'maxlags': 12, 'trend': 'c'}",-2795.219703,1204.302014,-5909.638512,8917.525276,6160.753496,11276.255871,0.058194,0.031725,0.073493,10561.908861,7445.873926,12409.500098
"{'ic': 'bic', 'maxlags': 12, 'trend': 'ct'}",-4073.326946,860.320188,-7778.014603,9473.802899,6045.226364,12320.321284,0.063141,0.032907,0.081941,11263.619578,7393.434022,13465.664309
"{'ic': 'fpe', 'maxlags': 12, 'trend': 'c'}",-3270.161413,-1369.422142,-4855.580896,8353.637979,5962.478198,9942.458542,0.059515,0.037735,0.072013,9980.495691,7060.344126,11328.148586
"{'ic': 'fpe', 'maxlags': 12, 'trend': 'ct'}",-1541.914090,-368.250579,-2776.816284,9000.169368,6832.816908,10363.483546,0.059654,0.038920,0.071897,10434.803440,7910.587793,11533.490446
"{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}",-2425.982698,-578.767991,-3984.639899,8401.430147,6447.117332,9771.827228,0.060639,0.039639,0.072748,10086.935039,7494.178038,11302.568653


In [29]:
best_specification_mx_no_trib = "{'ic': 'aic', 'maxlags': 12, 'trend': 'c'}"
best_prediction_mx_no_trib = {}
for split, predictions in prediction_mx_no_trib.items():
    best_prediction_mx_no_trib[split] = predictions[best_specification_mx_no_trib]

# Modelos Var de Ingresos Tributarios
Opciones de variables endógenas:
- isr_neto_(mdp)_r, iva_neto_(mdp)_r,
- isr_neto_(mdp)_r, iva_neto_(mdp)_r, importaccines_neto_mdp_r,
- isr_neto_(mdp)_r, iva_neto_(mdp)_r, ieps_sin_gasol_neto_(mdp)_r, importaccines_neto_mdp_r

In [51]:
# Predecimos con Modelo VAR
accuracies_var = []
accuracies_var_y = []
for model in params['ingresos_fiscales_var']:
    prediction_var, results_var, y_var_diffs = mv.run_predictions(
        model_name='VAR',
        params=all_models_params['VAR'],
        begin_and_ends=begin_and_ends,
        df=df_netos,
        endog_cols=model['endog_vars'],
        endog_transformation=model['endog_transformation'],
        endog_lags=None,
        exog=True,
        mex_prediction_dict=best_prediction_mx_w_trib,
        us_prediction_dict=best_prediction_us,
        exog_vars=model['exog_vars'],
        exog_transformation=model['exog_transformation'],
        other_exog=model['other_exog'],
        exog_pct_changes=None,
        exog_month_dummies=model['month_dummies'],
        exog_year_dummies=model['year_dummies'],
        exog_lags=None,
        train_start=model['start_var'])
    accuracies_var += results_var
    accuracies_var_y += y_var_diffs
results_var = pd.DataFrame(accuracies_var)
results_var.to_csv('../results/results_var_new.csv')

COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'ic': 'fpe', 'maxlags': 12, 'trend': 'c'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'ic': 'fpe', 'maxlags': 12, 'trend': 'ct'} for split 2015-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'c'} for split 2016-07-0

COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} for split 2016-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'c'} for split 2016-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} for split 2016-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'ic': 'fpe', 'maxlags': 12, 'trend': 'c'} for split 2016-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'ic': 'fpe', 'maxlags': 12, 'trend': 'ct'} for split 2016-07-01 00:00:00
COMPLETED VAR for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'ic': 'aic', 'maxlags': 1

In [53]:
y_var_diffs_df = pd.DataFrame(y_var_diffs)

In [54]:
y_var_diffs_df.to_csv('../results/precision_cierre_var_ing_trib.csv')

In [33]:
print(time.ctime())

Tue Aug 13 21:26:17 2019


In [34]:
results_var.groupby(['model', 'params', 'endog_vars', 'exog_vars', 'variable']).mean()\
    .sort_values('mape_first6')

forecast_biass_first18  \
model params                                      endog_vars                                         exog_vars                                          variable                                             
VAR   {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -5111.288641   
      {'ic': 'bic', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                       -787.865404   
      {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'ieps_... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -5752.126140   
      {'ic': 'bic', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'ieps_... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -2148.134807   
      {'ic': 'aic', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -3447.764306   
      {'ic': 'fpe', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -3447.764306   
      {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'impor... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                        245.211681   
      {'ic': 'fpe', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -6747.873299   
      {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -6747.873299   
      {'ic': 'bic', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'impor... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                       7168.083743   
                                                  ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r']           ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -2613.230282   
      {'ic': 'bic', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r']           ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -7110.893235   
      {'ic': 'fpe', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'impor... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -3002.482006   
      {'ic': 'aic', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'impor... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -3002.482006   
                                                  ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r']           ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -4294.417275   
      {'ic': 'fpe', 'maxlags': 12, 'trend': 'c'}  ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r']           ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -4294.417275   
      {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'impor... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -6368.538692   
      {'ic': 'fpe', 'maxlags': 12, 'trend': 'ct'} ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'impor... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... isr_neto_(mdp)_r                      -6368.538692   
      {'ic': 'aic', 'maxlags': 12, 'trend': 'ct'} ['isr_neto

# Modelo ARIMA y SARIMA
5 modelos:
- Ing trib sin gas
- IEPS sin gas
- ISR
- IVA
- Importaciones

In [37]:
# Predecimos con ARIMA y SARIMA
predictions_arima = {}
accuracies_arima= []
accuracies_arima_y = []
for model in params['ingresos_fiscales_arima']:
    for model_name in model['models']:
        predictions_arima[model_name] = predictions_arima.get(model_name, {})
        var = model['endog_vars'][0]
        predictions_arima[model_name][var] = {}
        for i, exog_var in enumerate(model['exog_vars']):
                if i <= 1:
                    continue
        preds, accurs, y_accurs = mv.run_predictions(
            model_name=model_name,
            params=all_models_params[model_name],
            begin_and_ends=begin_and_ends,
            df=df_netos,
            endog_cols=[var],
            endog_transformation=model['endog_transformation'],
            endog_lags=None,
            exog=True,
            mex_prediction_dict=best_prediction_mx_w_trib,
            us_prediction_dict=best_prediction_us,
            exog_vars = exog_var,
            exog_transformation=model['exog_transformation'],
            other_exog=semana_santa_y_tasas,
            exog_pct_changes=None,
            exog_month_dummies=model['month_dummies'],
            exog_year_dummies=model['year_dummies'],
            exog_lags=None,
            train_start=model['start_arima'])
        predictions_arima[model_name][var][str(exog_var)] = preds
        accuracies_arima += accurs
        accuracies_arima_y += y_accurs

COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (2, 0, 2)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (2, 0, 3)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (2, 0, 4)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (3, 0, 2)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (3, 0, 3)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (3, 0, 4)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (4, 0, 2)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (4, 0, 3)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for ['ing_trib_neto_(mdp)_r'] with params {'order': (4, 0, 4)} for split 2015-07-01 00:00:00
COMPLETED ARIMA for

COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (2, 0, 3), 'seasonal_order': (0, 0, 1, 12)} for split 2016-07-01 00:00:00
COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (2, 0, 3), 'seasonal_order': (1, 0, 0, 12)} for split 2016-07-01 00:00:00
COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (2, 0, 3), 'seasonal_order': (1, 0, 1, 12)} for split 2016-07-01 00:00:00
COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (3, 0, 2), 'seasonal_order': (0, 0, 0, 12)} for split 2016-07-01 00:00:00
COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (3, 0, 2), 'seasonal_order': (0, 0, 1, 12)} 

COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (3, 0, 3), 'seasonal_order': (0, 0, 1, 12)} for split 2018-07-01 00:00:00
COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (3, 0, 3), 'seasonal_order': (1, 0, 0, 12)} for split 2018-07-01 00:00:00
COMPLETED SARIMA for ['ing_trib_neto_(mdp)_r'] with params {'enforce_invertibility': False, 'enforce_stationarity': False, 'order': (3, 0, 3), 'seasonal_order': (1, 0, 1, 12)} for split 2018-07-01 00:00:00


In [38]:
print(time.ctime())

Tue Aug 13 21:59:27 2019


In [39]:
results_arima = pd.DataFrame(accuracies_arima)
results_arima.to_csv('../results/results_arima_new_ing_total.csv')
results_arima.groupby(['model', 'params', 'exog_vars', 'variable']).mean().sort_values('mape_first6')

forecast_biass_first18  \
model  params                                             exog_vars                                          variable                                        
SARIMA {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -10971.412388   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -10698.182195   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -18975.456449   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -21099.421170   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -16423.067936   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -21764.083844   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -19246.651588   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -20661.611697   
ARIMA  {'order': (2, 0, 4)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -11042.891629   
       {'order': (4, 0, 2)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -12485.674562   
       {'order': (3, 0, 3)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -13398.174895   
       {'order': (4, 0, 3)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -14038.842599   
       {'order': (3, 0, 2)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -12725.653788   
       {'order': (4, 0, 4)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -11359.964229   
       {'order': (3, 0, 4)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -12374.940025   
SARIMA {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -23475.570652   
ARIMA  {'order': (2, 0, 3)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -13960.048159   
SARIMA {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -23868.451261   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -22327.714710   
ARIMA  {'order': (2, 0, 2)}                               ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -15047.541067   
SARIMA {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -28455.132479   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -25856.284530   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -29838.871063   
       {'enforce_invertibility': False, 'enforce_stati... ['igae_sa', 'inpc', 'tasa_cetes_91_mensual', 't... ing_trib_neto_(mdp)_r           -33180.722222   
       {'enforce_inverti

In [40]:
accuracies_arima_y_df = pd.DataFrame(accuracies_arima_y)
accuracies_arima_y_df.to_csv('../results/precision_cierre_arima_ing_trib_ing_total.csv')

In [41]:
df_ml = pd.concat([df_netos, exportaciones['exportaciones_r'], igae_prim, igae_secun, igae_terc,
                   indic_adelant, imai_mineria, imai_construccion, imai_manufacturas,
                   imai_egergia_gas_agua_gas], axis=1)

In [42]:
vars_to_predict = df_ml.columns[df_ml.tail().isna().any()]
print(vars_to_predict)
for var in vars_to_predict:
    print(var)
    # Volvemos a obtener variables que tienen valores nulos, pues cada que predecimos una variable,
    # acutalizamos.
    empty_vars = df_ml.columns[df_ml.tail().isna().any()]
    vars_to_use_as_covars = [var for var in df_ml.columns if var not in empty_vars]
    exog = df_ml[vars_to_use_as_covars].copy()
    # Creamos variable de mes
    exog = mv.construct_exog(exog, month_dummies=True, year_dummies=[2000, 2009])
    prediction_start = df_ml.loc[df_ml[var].notna()].index.max() + relativedelta(months=1)
    _, prediction = mv.predict_with_econometric_model(
        model_name='SARIMA',
        params={'order': (3, 0, 3), 'seasonal_order': (1, 0, 1, 12),
                'enforce_stationarity': False, 'enforce_invertibility': False},
        df=df_ml,
        endog_vars=[var],
        prediction_start=prediction_start,
        prediction_end=df_ml.index.max(),
        transformation='log_diff',
        lags_endog=None,
        exog_df=exog,
        train_start=None)
    df_ml.loc[pd.date_range(prediction_start, df_ml.index.max(), freq='MS'), var] = prediction
    print('Nowcasted {}'.format(var))

Index(['igae_prim', 'igae_secun', 'igae_terc', 'indic_adelant'], dtype='object')
igae_prim
Nowcasted igae_prim
igae_secun
Nowcasted igae_secun
igae_terc
Nowcasted igae_terc
indic_adelant
Nowcasted indic_adelant


In [43]:
df_ml = df_ml.loc[df_ml.notna().all(1)]

In [44]:
params['ml_model']

[{'models': ['GB'],
  'endog_vars': ['isr_neto_(mdp)_r',
   'iva_neto_(mdp)_r',
   'importaciones_neto_(mdp)_r'],
  'endog_transformation': 'log_diff',
  'endog_lags': range(1, 13),
  'year_dummies': True,
  'exog_transformation': 'log_diff',
  'exog_pct_changes': [1, 12],
  'exog_lags': range(1, 13),
  'month_dummies': True,
  'start': '1990-01-01'},
 {'models': ['GB'],
  'endog_vars': ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'],
  'endog_transformation': 'log_diff',
  'endog_lags': range(1, 13),
  'year_dummies': True,
  'exog_transformation': 'log_diff',
  'exog_pct_changes': [1, 12],
  'exog_lags': range(1, 13),
  'month_dummies': True,
  'start': '1990-01-01'},
 {'models': ['GB', 'RF'],
  'endog_vars': ['iva_neto_(mdp)_r'],
  'endog_transformation': 'log_diff',
  'endog_lags': range(1, 13),
  'year_dummies': True,
  'exog_transformation': 'log_diff',
  'exog_pct_changes': [1, 12],
  'exog_lags': range(1, 13),
  'month_dummies': True,
  'start': '1990-01-01'},
 {'models': ['GB'],
  'e

In [45]:
# Vamos a correr modelos de ML
predictions_ml = {}
accuracies_ml = []
accuracies_ml_y = []
for model in params['ml_model']:
    predictions_ml[str(model['endog_vars'])] = {}  
    for model_name in model['models']:
        preds, accurs, y_accurs = mv.run_predictions(
            model_name=model_name,
            params=all_models_params[model_name],
            begin_and_ends=begin_and_ends,
            df=df_ml,
            endog_cols=model['endog_vars'],
            endog_transformation=model['endog_transformation'],
            endog_lags=model['endog_lags'],
            exog=True,
            mex_prediction_dict=best_prediction_mx_no_trib,
            us_prediction_dict=best_prediction_us,
            exog_vars = None,
            exog_transformation=model['exog_transformation'],
            other_exog=semana_santa_y_tasas,
            exog_pct_changes=model['exog_pct_changes'],
            exog_month_dummies=model['month_dummies'],
            exog_year_dummies=model['year_dummies'],
            exog_lags=model['exog_lags'],
            train_start=model['start'])
        predictions_ml[str(model['endog_vars'])][model_name] = preds
        accuracies_ml += accurs
        accuracies_ml_y += y_accurs
        results_ml = pd.DataFrame(accuracies_ml)
        results_ml.to_csv('../results/results_ML_new_1000.csv')


COMPLETED GB for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 1000, 'random_state': 1234, 'subsample': 0.5} for split 2015-07-01 00:00:00
COMPLETED GB for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 1000, 'random_state': 1234, 'subsample': 0.5} for split 2016-07-01 00:00:00
COMPLETED GB for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 1000, 'random_state': 1234, 'subsample': 0.5} for split 2017-07-01 00:00:00
COMPLETED GB for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r', 'importaciones_neto_(mdp)_r'] with params {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 1000, 'random_state': 1234, 'subsample': 0.5} for split 2018-07-01 00:00:00
COMPLETED GB for ['isr_neto_(mdp)_r', 'iva_neto_(mdp)_r'] with params {'lear

In [46]:
results_ml = pd.DataFrame(accuracies_ml)
results_ml.to_csv('../results/results_ML_new_1000.csv')

In [47]:
accuracies_ml_y_df = pd.DataFrame(accuracies_ml_y)
accuracies_ml_y_df.to_csv('../results/estimacion_cierre_ml_modelostotales.csv')